## SRJ - GCIM - Repetitive operators

In [1]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
print('GCIM with repetitive operators')
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
grad_GCIM = []  # To store the highest gradient excitation values
operator_check = []  # To store the highest gradient excitation operators
theta = np.pi/4
print('Theta is', theta)

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886 
#r_bohr = bond_length * 1.0  # Convert bond length to Bohr radius 
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
print(coordinates)
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[240] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    #print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.DoubleExcitation(theta, wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                qml.SingleExcitation(theta, wires=ash_excitation[i])
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
            qml.DoubleExcitation(theta, wires=ash_excitation)
        elif len(ash_excitation) == 2:
            qml.SingleExcitation(theta, wires=ash_excitation)
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

    
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('The operator pool length is', len(operator_pool))
    states = [hf_state]
    max_operator = None

    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        #max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        counts1 = measure(k)  #Measure the current state
        print('The measurement of new state is', counts1)
        print(f'The operator check for this iteration {j} is', operator_check)
        for i in operator_pool:
            #print('The current excitation operator is', i) #Current excitation operator - fermionic one
            #print('The value of  operator check is', operator_check)  #Max excitation operator
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)
        grad_GCIM.append(max_value)  # Append the highest operator to the grad_GCIM list
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
        else:
            print('Skipping individual excitation state for j < 2')
        states.append(ostate)
        print(f'After adding the states the adapt iteration is {j} and no of states',len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))  # Get the Hamiltonian matrix
        #print('Shape of Hamiltonian matrix is', Ham_matrix.shape)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                #print('The value of i and j is', i, j)
                left_op = states[i].T.conj()
                right_op = states[j]
                M[i,j] = left_op.dot(Ham_matrix.dot(right_op)).real
                S[i,j] = left_op.dot(right_op).real
     
                
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        #print(f'S matrix with {i} and {j} is', S)
        #print('Hamiltonian Matrix with', i, j, 'is', M)
        #print('The shape of S matrix is', S.shape)  
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy) 
          
    return ash_excitation, states,eig,gs_energy, Ham_matrix,hf_state, grad_GCIM


GCIM with repetitive operators
Theta is 0.7853981633974483
[[ 0.          0.          5.66917797]
 [ 0.          0.         11.33835593]
 [ 0.          0.         17.0075339 ]
 [ 0.          0.         22.67671186]]


In [2]:
ash_excitation, states,eig,gs_energy,Ham_matrix, hf_state, grad_GCIM = adaptvqe(adapt_it=26, e_th=1e-12)
print('The final ash_excitation is', ash_excitation)
print('GS energy is', gs_energy)
print('gradient GCIM is', grad_GCIM)

HF state is -1.3325688880668918
The operator pool length is 26
The adapt iteration now is 1
The measurement of new state is {'11110000': 1000}
The operator check for this iteration 1 is []
The expectation value of a⁺(0) a(4) is 9.258183415045138e-08
The expectation value of a⁺(0) a(6) is 1.5070623234594148e-49
The expectation value of a⁺(1) a(5) is 9.25818341226958e-08
The expectation value of a⁺(1) a(7) is 0.0
The expectation value of a⁺(2) a(4) is 1.6751879955323474e-49
The expectation value of a⁺(2) a(6) is 1.169896999275721e-07
The expectation value of a⁺(3) a(5) is 0.0
The expectation value of a⁺(3) a(7) is 1.1698969991369432e-07
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.3057475259056449
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 0.0
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 0.0
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.2841068609808367
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.030544625693071533
The expectation value of a⁺(0) a⁺(3) a(4

In [3]:
gs_energy

[-1.430083670535935,
 -1.5694603832062615,
 -1.7440626565880086,
 -1.8660667753404518,
 -1.8660667754900027,
 -1.866066775527674,
 -1.8695103866505207,
 -1.8758554606016085,
 -1.879996030083914,
 -1.8799960302584826,
 -1.8799960302968957,
 -1.8805909454374903,
 -1.880590945580199,
 -1.8819740553489446,
 -1.8819740555862778,
 -1.8819740556076343,
 -1.8819740556155653,
 -1.881974055853086,
 -1.8841978746690582,
 -1.884197874685723,
 -1.884197874690641,
 -1.8844641349770057,
 -1.8844642247225247,
 -1.8844642285607307,
 -1.8844642351402527]

In [4]:
grad_GCIM

[0.31004515885821066,
 0.3658436432870543,
 0.415301903154508,
 0.4355198437800792,
 0.21111995508844042,
 0.22815479832037758,
 0.11777814045647549,
 0.13192141141184435,
 0.11249432466754647,
 0.10990279126279166,
 0.09100384851899943,
 0.0676229706602161,
 0.12233660211094906,
 0.11622221048972657,
 0.13826533693112636,
 0.14306468561960603,
 0.12341855506396746,
 0.12411222972736854,
 0.1440220480917352,
 0.2100679334697848,
 0.26765067526978886,
 0.3175788812293109,
 0.3133502160325448,
 0.2784206982007126,
 0.2191015679636771]